In [1]:
from prody import *
from pylab import *
import json
from statistics import mean

In [2]:
pdb, header = parsePDB('6acg', header=True)

@> PDB file is found in working directory (6acg.pdb).
@> 29715 atoms and 1 coordinate set(s) were parsed in 0.33s.


In [3]:
# get contacts
spike_contacts = pdb.select('chain A B C and within 5 of chain D')
ace2_contacts = pdb.select('chain D and within 5 of chain A B C')

# writePDB('spike_contacts.pdb', spike_contacts)
# writePDB('ace2_contacts.pdb', ace2_contacts)

In [4]:
spike = pdb.select('chain A B C')
ace2  = pdb.select('chain D')

In [5]:
interactions = findNeighbors(atoms=spike, radius=5, atoms2=ace2)

In [6]:
# data structures to store nodes and links
# dictionaries used to avoid duplicates
# converted to lists later

spike_atoms = {}
ace2_atoms = {}

atom_links = []

spike_residues = {}
ace2_residues = {}

residue_links = {} 
# dictionary because mutliple links possible
# just increment count instead

In [7]:
# function to format atom data
def getAtomData(atom):
    data = {}

    data["id"]       = int(atom.getIndex())
    data["atom"]     = atom.getData('name')
    data["chain"]    = atom.getData('chain')
    data["resname"]  = atom.getData('resname')
    data["resnum"]   = int(atom.getData('resnum'))
    data["resindex"] = int(atom.getData('resindex'))
    data["name"]     = " ".join([data["chain"], data['resname'], str(data['resnum']), data['atom']])
    data["bonds"]    = 1 # number of atoms this atom is bonded with

    data['fx'], data['fy'], data['fz'] = tuple(atom.getCoords())
    data['fx'] = float(data['fx'])
    data['fy'] = float(data['fy'])
    data['fz'] = float(data['fz'])
    
    
    return data

# function to get residue data
def getResidueData(residue):
    # getting a Residue involves making a Hierarchial View
    # which is a lot of computation
    # so it's easier to get the data from any one atom
    atom = residue[0]

    data = {}

    data['id'] = int(atom.getData('resindex'))
    data['chain'] = atom.getData('chain')
    data['resnum'] = int(atom.getData('resnum'))
    data['aa'] = atom.getData('resname')
    data['name'] = data['chain'] + str(data['resnum']) + '_' + data['aa']
    
    coordinates_list = residue.getCoords()
    data['fx'] = float(mean([coordinates[0] for coordinates in coordinates_list]))
    data['fy'] = float(mean([coordinates[1] for coordinates in coordinates_list]))
    data['fz'] = float(mean([coordinates[2] for coordinates in coordinates_list]))

    data['bonds'] = 1 # number of atoms this residue's atoms are bonded with`

    return data


In [8]:
# store nodes and links 
for pair in interactions:
    a, b, distance = pair

    assert(a.getChid() == 'C')
    assert(b.getChid() == 'D')

    spike_atom_ID = a.getIndex()
    ace2_atom_ID  = b.getIndex()

    spike_residue_ID = a.getResindex()
    ace2_residue_ID  = b.getResindex()
    
    # check if atom is already present
    if spike_atom_ID not in spike_atoms.keys():
        spike_atom = getAtomData(a)
        spike_atoms[spike_atom_ID] = spike_atom

        # if atom is present, residue must be present,
        # no point of checking it outside this if branch
        if spike_residue_ID not in spike_residues.keys():
            spike_residue = pdb.select('resindex ' + str(spike_residue_ID))
            spike_residue = getResidueData(spike_residue)
            spike_residues[spike_residue_ID] = spike_residue
    else:
        # increment number of connections for atom and residue
        spike_atoms[spike_atom_ID]['bonds'] += 1
        spike_residues[spike_residue_ID]['bonds'] += 1

    if ace2_atom_ID not in ace2_atoms.keys():
        ace2_atom = getAtomData(a)
        ace2_atoms[ace2_atom_ID] = ace2_atom

        if ace2_residue_ID not in ace2_residues.keys():
            ace2_residue = pdb.select('resindex ' + str(ace2_residue_ID))
            ace2_residue = getResidueData(ace2_residue)
            ace2_residues[ace2_residue['id']] = ace2_residue
    else:
        ace2_atoms[ace2_atom_ID]['bonds'] += 1
        ace2_residues[ace2_residue_ID]['bonds'] += 1

    atom_links.append({
        "source": spike_atom['id'],
        "target": ace2_atom['id'],
        "distance": float(distance)
    })

    if (spike_residue_ID, ace2_residue_ID) in residue_links.keys():
        residue_links[(spike_residue_ID, ace2_residue_ID)]['strength'] += 1
    else:
        residue_links[(spike_residue_ID, ace2_residue_ID)] = {
            "source":   int(spike_residue_ID),
            "target":   int(ace2_residue_ID),
            "strength": 1
        }

In [9]:
# sort nodes by bonds
spike_atoms = sorted(spike_atoms.values(), key = (lambda i : i['bonds']), reverse=True)
ace2_atoms  = sorted(ace2_atoms.values(),  key = (lambda i : i['bonds']), reverse=False)

spike_residues = sorted(spike_residues.values(), key = (lambda i : i['bonds']), reverse=True)
ace2_residues  = sorted(ace2_residues.values(),  key = (lambda i : i['bonds']), reverse=False)

# sort links by distance and strength
atom_links = sorted(atom_links, key = (lambda i : i['distance']), reverse=False)
residue_links = sorted(residue_links.values(), key = (lambda i : i['strength']), reverse=True)


In [10]:
x_sum, y_sum, z_sum = 0, 0, 0
for atom in spike_atoms + ace2_atoms:
    x_sum += atom['fx']
    y_sum += atom['fy']
    z_sum += atom['fz']
n = len(spike_atoms) + len(ace2_atoms)
x_avg, y_avg, z_avg = x_sum / n, y_sum / n, z_sum / n

print(x_avg, y_avg, z_avg)

for atom in spike_atoms + ace2_atoms:
    atom['fx'] = atom['fx'] - x_avg
    atom['fy'] = atom['fy'] - y_avg
    atom['fz'] = atom['fz'] - z_avg

for residue in spike_residues + ace2_residues:
    residue['fx'] = residue['fx'] - x_avg
    residue['fy'] = residue['fy'] - y_avg
    residue['fz'] = residue['fz'] - z_avg

210.16135204081633 205.19093877551018 119.44444387755088


In [11]:
x_sum, y_sum, z_sum = 0, 0, 0
for residue in spike_residues + ace2_residues:
    x_sum += residue['fx']
    y_sum += residue['fy']
    z_sum += residue['fz']
n = len(spike_residues) + len(ace2_residues)
x_avg, y_avg, z_avg = x_sum / n, y_sum / n, z_sum / n

print(x_avg, y_avg, z_avg)

-1.006574544123204 0.5514214930985286 -0.9670664785609819


In [12]:
atom_interactions = {
    "nodes": spike_atoms + ace2_atoms,
    "links": atom_links
}

residue_interactions = {
    "nodes": spike_residues + ace2_residues,
    "links": residue_links
}

In [13]:
# write to json
with open('3d_atom_interactions.json', 'w') as file:
    json.dump(atom_interactions, file, indent=2)

In [14]:
hello = residue_interactions['links'][0]['source']

print(type(hello))
print(hello)

<class 'int'>
2595


In [15]:
with open('3d_residue_interactions.json', 'w') as file:
    json.dump(residue_interactions, file, indent=2)